In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind

base = pd.read_csv('main_task.csv')

col = ['Restaurant_id', 'City', 'Cuisine_Style', 'Ranking',
       'Rating', 'Price_Range', 'Number_of_Reviews', 'Reviews', 'URL_TA', 'ID_TA']
base.columns = col
#переименовали столбы для удобства

#base=base.drop(index=[9446])
#удалили строчку, состоящую полностью из ошибок

base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine_Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price_Range        26114 non-null  object 
 6   Number_of_Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [14]:
# проверяем столбец с Restaurant_id (идентификационный номер ресторана / сети ресторанов)

base.Restaurant_id.value_counts()
# пропусков нет, все ок
base.Restaurant_id.describe()
# по условию - в одному id  могут быть прикреплены несколько ресторанов одной сети,
# поэтому уникальных значений меньше чем записей
len(base.Restaurant_id)

# аномалий нет

40000

In [15]:
# проверяем столбец с City (город, в котором находится ресторан)

base.City.value_counts()
len(base.City)
# пропусков нет, города не повторяются, проблем с ракладкой или регистром нет, все ок

# аномалий нет

40000

In [16]:
# проверяем столбец с Cuisine_Style (кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане)

base['Cuisine_Style'] = base['Cuisine_Style'].fillna('""No_info"')
# заменяем пропуски

new = pd.DataFrame(base.Cuisine_Style.dropna())
a = list(new.Cuisine_Style)
b = list()


def l(x):
    i = 0
    for g in x:
        f = x[i].split(',')
        v = 0
        for g in f:
            h = f[v][2:-1].replace("'", '')
            v = +1
            b.append(h)
        i += 1


l(a)
b = set(b)
b = list(b)
# нашли список уникальных значений кухонь


def find_item(cell):
    if item in cell:
        return 1
    return 0


for item in b:
    base[item] = base['Cuisine_Style'].apply(find_item)

base['Cuisine_Style'] = base['Cuisine_Style'].apply(lambda x: len(x))

len(base['Cuisine_Style'])
# пропуски заменены, строки преобразованы в столбцы

40000

In [5]:
# проверяем столбец с Price_Range (диапазон цен в ресторане)

base['Price_Range'] = base['Price_Range'].fillna('No_info')
len(base['Price_Range'])

# меняем пропуски на Not_info

40000

In [27]:
# проверяем столбец с Reviews (данные о двух отзывах, которые отображаются на сайте ресторана)

base['Reviews']=base.Reviews.replace("[[], []]", 'No_info')
len(base.Reviews)

# меняем пропуски на Not_info

40000

In [28]:
# проверяем столбец с Number_of_Reviews (количество отзывов о ресторане)
base['Number_of_Reviews'].describe()
# отрицательных чисел нет

base['Number_of_Reviews'].value_counts()
# но есть пропуски, нужно проверить, что где есть пропуски - там действительно нет отзывов

base['Number_of_Reviews'] = base['Number_of_Reviews'].fillna('No_info')
# меняем пропуски на Not_info

a=base.query('Reviews == "No_info" & Number_of_Reviews == "No_info"')
b=list(a.index)
base['Number_of_Reviews'][b]=0
# где нет значения кол-ва отзывов и превью отзывов - ставим 0

len(base['Number_of_Reviews'])
# заменили предполагаемые пропуски, в некоторые смогли подставить значения. С 2,5+ тысяч снизили пропуски до 900+

/Users/yasckevich/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


40000

In [8]:
# проверяем м столбец с URL_TA (URL страницы ресторана на TripAdvosor)

base['URL_TA'].value_counts()

# очень странно, что сслылки повторяются
# посмотрим что с ними на примере одного ресторана
base.query(
    'URL_TA == "/Restaurant_Review-g187514-d697406-Reviews-Al_Mounia-Madrid.html"')
# совпадают 8 столбцов из 10 основных

# удалим дубликаты по стобцам сайт, ID на сайте, город, кол-во отзывов, кухни и средний чек
base = base.drop_duplicates(
    subset=['City', 'Reviews', 'URL_TA', 'ID_TA', 'Cuisine_Style', 'Price_Range'])

base['URL_TA'].value_counts()

# дубликаты удалены

/Restaurant_Review-g187323-d6387209-Reviews-Su_Korean_BBQ-Berlin.html                        1
/Restaurant_Review-g187791-d3337115-Reviews-Cucina_Casereccia_da_Marcello-Rome_Lazio.html    1
/Restaurant_Review-g187514-d10404717-Reviews-Sota_Caballo_Rey-Madrid.html                    1
/Restaurant_Review-g187323-d8397770-Reviews-Fratelli_Da_Giovanni-Berlin.html                 1
/Restaurant_Review-g187147-d719440-Reviews-Le_Tabarin-Paris_Ile_de_France.html               1
                                                                                            ..
/Restaurant_Review-g187309-d7047665-Reviews-IceDate-Munich_Upper_Bavaria_Bavaria.html        1
/Restaurant_Review-g186338-d7741223-Reviews-Made_in_South_Pizzeria-London_England.html       1
/Restaurant_Review-g187514-d9609663-Reviews-El_Murciano_Sirod-Madrid.html                    1
/Restaurant_Review-g187147-d718162-Reviews-L_Epicerie-Paris_Ile_de_France.html               1
/Restaurant_Review-g187791-d12481595-Reviews-Finge

In [9]:
# проверяем м столбец с ID_TA (идентификатор ресторана в базе данных TripAdvisor)

base['ID_TA'].value_counts()

# так как в предыдущем шаге мы удалили все дубликаты, то в этом у нас все ок

d3707258     1
d998025      1
d3777421     1
d7156311     1
d4129783     1
            ..
d5122124     1
d11832263    1
d10171093    1
d10202812    1
d5518627     1
Name: ID_TA, Length: 39980, dtype: int64

In [10]:
df = base.loc[:, col]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39980 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      39980 non-null  object 
 1   City               39980 non-null  object 
 2   Cuisine_Style      39980 non-null  int64  
 3   Ranking            39980 non-null  float64
 4   Rating             39980 non-null  float64
 5   Price_Range        39980 non-null  object 
 6   Number_of_Reviews  39980 non-null  object 
 7   Reviews            39980 non-null  object 
 8   URL_TA             39980 non-null  object 
 9   ID_TA              39980 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 3.4+ MB
